In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np

In [174]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [175]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': '',
 'current_data': (datetime.datetime(2022, 9, 7, 11, 36, 49, 310651),
                timeStamp  value      fuelType
  0   2022-09-07 00:00:00   1256          Coal
  1   2022-09-07 00:05:00   1260          Coal
  2   2022-09-07 00:10:00   1256          Coal
  3   2022-09-07 00:15:00   1257          Coal
  4   2022-09-07 00:20:00   1261          Coal
  ..                  ...    ...           ...
  135 2022-09-07 11:15:00    -59  Saskatchewan
  136 2022-09-07 11:20:00    -59  Saskatchewan
  137 2022-09-07 11:25:00    -59  Saskatchewan
  138 2022-09-07 11:30:00    -58  Saskatchewan
  139 2022-09-07 11:35:00    -58  Saskatchewan
  
  [1540 rows x 3 columns]),
 'daily_outage_dfs': [(datetime.datetime(2022, 9, 4, 8, 42, 44, 922164, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                       timeStamp  value  fuelType
   0   2022-09-02 00:00:00-06:00    290  Intertie
   1   2022-09-03 00:00:00-06:00      0  Intertie
   2   2022-09-04 00:00:00-06:

In [176]:
default_pickle['monthly_outage_dfs'][0]

(datetime.datetime(2022, 9, 4, 8, 42, 44, 922164, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp  value         fuelType
 9  2022-10-01 00:00:00-06:00    470             Coal
 10 2022-11-01 00:00:00-06:00     30             Coal
 11 2022-12-01 00:00:00-07:00      0             Coal
 9  2022-10-01 00:00:00-06:00   1130      Natural Gas
 10 2022-11-01 00:00:00-06:00    740      Natural Gas
 ..                       ...    ...              ...
 4  2024-05-01 00:00:00-06:00     30  Biomass & Other
 5  2024-06-01 00:00:00-06:00     30  Biomass & Other
 6  2024-07-01 00:00:00-06:00     30  Biomass & Other
 7  2024-08-01 00:00:00-06:00     30  Biomass & Other
 8  2024-09-01 00:00:00-06:00     30  Biomass & Other
 
 [192 rows x 3 columns])

In [200]:
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
print(daily)
# monthly['timeStamp'] = monthly['timeStamp'].dt.tz_localize(tz='America/Edmonton')
# print(monthly)
# with open('./monthly_df.pickle', 'wb') as handle:
#     pickle.dump(monthly, handle, protocol=pickle.HIGHEST_PROTOCOL)

                    timeStamp  value  fuelType
0   2022-09-02 00:00:00-06:00    290  Intertie
1   2022-09-03 00:00:00-06:00      0  Intertie
2   2022-09-04 00:00:00-06:00      0  Intertie
3   2022-09-05 00:00:00-06:00      0  Intertie
4   2022-09-06 00:00:00-06:00      0  Intertie
..                        ...    ...       ...
115 2022-12-26 00:00:00-07:00      0      Wind
116 2022-12-27 00:00:00-07:00      0      Wind
117 2022-12-28 00:00:00-07:00      0      Wind
118 2022-12-29 00:00:00-07:00      0      Wind
119 2022-12-30 00:00:00-07:00      0      Wind

[1323 rows x 3 columns]


In [201]:
a = daily[daily['timeStamp'].dt.date < datetime.now(pytz.timezone('America/Edmonton')).date() + timedelta(days=90)]